# Introduction #

In this code I will generate a chatbot trained with some content, ultimately expert in Rajas of the Ganges eurogame (board game).

In [1]:
!pip uninstall protobuf -y
!pip install protobuf==3.20.3
!pip install tensorflow==2.17.1 tensorflow-metadata==1.13.1 protobuf==3.20.3 chromadb==0.5.23
!pip install chromadb sentence-transformers

!pip install gdown
!pip install PyPDF2
!pip install youtube-transcript-api
!pip install langchain
!pip install -U langchain-community

!apt-get remove -y chromium-browser chromium-chromedriver
!rm -f /usr/bin/chromedriver

!pip install selenium
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver
!ln -s /usr/lib/chromium-browser/chromedriver /usr/bin/chromedriver

!pip install wikidata
!pip install langdetect
!pip install deep_translator

!pip install llama-index --upgrade
!pip install sentence-transformers --upgrade

!pip install python-decouple==3.8 llm-templates

Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3
  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (679 bytes)
Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.24
    Uninstalling langchain-core-0.3.24:
      Successfully uninstalled langchain-core-0.3.24
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.11
    Uninstalling langchain-0.3.1

# Step 1: Content Download #

## Text context: game rules ##

### Download PDF ###

In [2]:
import gdown
import os
import shutil

drive_folder_url = "https://drive.google.com/drive/folders/1bMK7C8qbU96FvUjOBT1D35wy1LZw3K4E?usp=sharing"

# Descarga carpeta 'game_data'
gdown.download_folder(drive_folder_url, quiet=True, output='game_data')

['game_data/rules_EN.pdf',
 'game_data/Rajas_of_the_Ganges_Plain_and_Simple.pdf',
 'game_data/Rajas_of_the_Ganges_Quick_Rules_Guide.pdf',
 'game_data/Rajas_of_the_Ganges_-_rules_in_brief_v1.pdf']

Extraer texto

In [3]:
import re

def remove_uppercase(x):
    x1 = x.lower()
    return x1

def remove_signs(x):
    x1 = re.sub(r'[^\w\s]', '', x)  # Elimina los signos, dejando solo palabras y espacios
    return x1.strip()

def remove_spaces(x):
    x1 = re.sub(r'\s+', ' ', x).strip()  # Reemplaza múltiples espacios con uno solo
    return x1

def clean_text(text):
    # Eliminar tabulaciones y dividir en oraciones
    text = text.replace("\t", "") #elimino tabs
    text = text.replace("\n", " ") #cambio saltos de línea por espacios

    # Aplicar las transformaciones en el orden correcto
    output = remove_uppercase(remove_signs(remove_spaces(text)))

    return output

In [4]:
import PyPDF2

# Abre el archivo PDF
pdf_path = 'game_data/Rajas_of_the_Ganges_-_rules_in_brief_v1.pdf'
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    context1 = ''

    # Extrae el texto de cada página
    for page in reader.pages:
        context1 += page.extract_text()

In [5]:
doc1 = clean_text(context1)
print(f"Longitud de doc1: {len(doc1)}")

Longitud de doc1: 3542


In [6]:
import PyPDF2

# Abre el archivo PDF
pdf_path = 'game_data/Rajas_of_the_Ganges_Plain_and_Simple.pdf'
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    context2 = ''

    # Extrae el texto de cada página
    for page in reader.pages:
        context2 += page.extract_text()

In [7]:
doc2 = clean_text(context2)
print(f"Longitud de doc2: {len(doc2)}")

Longitud de doc2: 13169


In [8]:
import PyPDF2

# Abre el archivo PDF
pdf_path = 'game_data/Rajas_of_the_Ganges_Quick_Rules_Guide.pdf'
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    context3 = ''

    # Extrae el texto de cada página
    for page in reader.pages:
        context3 += page.extract_text()

In [9]:
doc3 = clean_text(context3)
print(f"Longitud de doc3: {len(doc3)}")

Longitud de doc3: 13815


In [10]:
import PyPDF2

# Abre el archivo PDF
pdf_path = 'game_data/rules_EN.pdf'
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    context4 = ''

    # Extrae el texto de cada página
    for page in reader.pages:
        context4 += page.extract_text()

In [11]:
doc4 = clean_text(context4)
print(f"Longitud de doc4: {len(doc4)}")

Longitud de doc4: 35431


### Download YouTube video ###

In [12]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id_1 = "OJ0anCDdIJ4"
video_id_2 = "ruScGyu1_5w"

# Obtener el transcript
try:
    transcript1 = YouTubeTranscriptApi.get_transcript(video_id_1)
    transcript2 = YouTubeTranscriptApi.get_transcript(video_id_2)

    # Convertir el transcript en una sola cadena de texto
    transcript_text_1 = " ".join([entry["text"] for entry in transcript1])
    transcript_text_2 = " ".join([entry["text"] for entry in transcript2])

    context5 = transcript_text_1
    context6 = transcript_text_2
except Exception as e:
    print(f"Error al obtener el transcript: {e}")

In [13]:
doc5 = clean_text(context5)
print(f"Longitud de doc5: {len(doc5)}")

doc6 = clean_text(context6)
print(f"Longitud de doc6: {len(doc6)}")

Longitud de doc5: 11173
Longitud de doc6: 65091


### Chunking ###

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Concatenar todo el contexto en un solo texto
docs = [doc1, doc2, doc3, doc4, doc5, doc6]
texts, metadatas, ids = [], [], []

for i in range(len(docs)):
  # Dividir el texto en chunks
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
  chunks = text_splitter.split_text(docs[i])

  for chunk in chunks:
    texts.append(chunk)
    if i<5:
      metadatas.append({"document":f"doc{i+1}", "type":"rules"})
    else:
      metadatas.append({"document":f"doc{i+1}", "type":"review"})
    ids.append(f"doc{i+1}_{chunks.index(chunk)}")

print(len(texts))
print(len(metadatas))
print(len(ids))

for i in range(10):
  print(texts[i])

318
318
318
rajas of the ganges 2017 24 players designer inka brand  marcus brand duration  players money and fame markers cross 45 75 minutes board setup  pick the game board side for  of players  2p or 3p4p 3p  use cover tiles to cover marked spaces  sort province tiles by color then by symbol snake cow tiger to get 12 shuffled stacks  shuffle 8 white yield tiles and set face down stack on the temple player setup  each player gets a province board  kali statue board  6 workers  5 cubes  1 money marker  1
board  6 workers  5 cubes  1 money marker  1 fame marker  and 1 bonus marker for the money track  place 3 total workers on the fame track 15 space money track 20 space and river on bridge  place boat on starting space of the river put karma cube of level 1 of karma path put 1 upgrade cube in left column value 2 of each building development scroll  put 1 fame marker on space 0 of fame track put bonus marker with the river space side up on money tra ck space 12  each player takes and r

### Chroma DB ###

In [15]:
import chromadb
from sentence_transformers import SentenceTransformer

# BBDD
client = chromadb.Client()
try:
  client.delete_collection('vector_db')
except Exception:
  pass
vector_db = client.create_collection("vector_db")

# Embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Iteration over chunks
# texts metadatas ids
for i in range(len(texts)):
  embedding = embedding_model.encode(texts[i])
  vector_db.add(documents=[texts[i]], metadatas=[metadatas[i]], ids=[ids[i]], embeddings=[embedding])
  print(f"Id: {ids[i]} - Embedding length: {len(embedding)} - Metadata: {metadatas[i]}")


# TESTING
results = vector_db.query(query_texts=["players"], n_results=1)
print(results)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Id: doc1_0 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_1 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_2 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_3 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_4 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_5 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_6 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_7 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc2_0 - Embedding length: 384 - Metadata: {'document': 'doc2', 'type': 'rules'}
Id: doc2_1 - Embedding length: 384 - Metadata: {'document': 'doc2', 'type': 'rules'}
Id: doc2_2 - Embedding length: 384 - Metadata: {'document': 'doc2', 'type': 'rules'}
Id: doc2_3 - Embedding length: 384 - Metadata: {'document': 'doc2

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 98.9MiB/s]


{'ids': [['doc1_7']], 'embeddings': None, 'documents': [['mogul they are the new start player otherwise pass the marker clockwise take worker s back and start round end game  scoring  when 1 player has money and fame markers on same spot or pass each other the game ends players up to the starting player get another turn equal turns  winner is player with greatest difference between crossed markers  tie tied player who crossed 1st wins']], 'uris': None, 'data': None, 'metadatas': [[{'document': 'doc1', 'type': 'rules'}]], 'distances': [[1.0929194688796997]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [16]:
results = vector_db.query(query_texts=["amount of players"], n_results=1)
print(results)

{'ids': [['doc6_135']], 'embeddings': None, 'documents': [['on the opposite side is for three and four just because of some of the spaces theres a little bit more space right and so in terms of player count how do you feel uh plays well at all player counts yeah we played at all players we have played at all player accounts uh where is my favorite maybe three i want to say maybe uh it plays very well at two though its nice to be able to go you know back to back i will say that when higher player counts theres the same amount of tiles out there and so']], 'uris': None, 'data': None, 'metadatas': [[{'document': 'doc6', 'type': 'review'}]], 'distances': [[0.9863834977149963]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


## Dataframe context: game stats##

### Selenium setup ###

Eliminar conflictos de versiones

Instalar driver

### Download Game Stats ###

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Configuración de las opciones de Selenium para Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar sin interfaz gráfica
chrome_options.add_argument("--no-sandbox")  # Para entornos sin GUI
chrome_options.add_argument("--disable-dev-shm-usage")  # Para evitar problemas de memoria
chrome_options.binary_location = '/usr/bin/chromium-browser'  # Ruta al binario de Chromium

# Crear el driver de Selenium
driver = webdriver.Chrome(options=chrome_options)

# URL de la página a scrapeo
url = "https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges/stats"
driver.get(url)

# Esperar a que se cargue el contenido dinámico
time.sleep(3)

# Encontrar los elementos en la página
titles_elements = driver.find_elements(By.CLASS_NAME, 'outline-item-title')
descriptions_elements = driver.find_elements(By.CLASS_NAME, 'outline-item-description')

# Extraer texto
titles = [title.text.strip() for title in titles_elements]
descriptions = [description.text.strip() for description in descriptions_elements]

# Crear un DataFrame con los datos
data = {'Title': titles, 'Description': descriptions}
game_stats = pd.DataFrame(data)

print(game_stats)

# Cerrar el navegador
driver.quit()


             Title                      Description
0      Avg. Rating                            7.731
1   No. of Ratings                           14,698
2   Std. Deviation                             1.13
3           Weight                         2.89 / 5
4         Comments                            2,124
5             Fans                              691
6       Page Views                          899,384
7     Overall Rank             156\nHistorical Rank
8    Strategy Rank             123\nHistorical Rank
9   All Time Plays                           59,431
10      This Month                              222
11             Own                           19,555
12     Prev. Owned                            2,055
13       For Trade      194\nFind For-Trade Matches
14   Want In Trade  677\nFind Want-in-Trade Matches
15        Wishlist                            3,812
16       Has Parts                               11
17      Want Parts                                4


## Knowledge Graph context: related games ##

In [18]:
import requests

def get_wikidata_info(entity_id):
    sparql_endpoint = "https://query.wikidata.org/sparql"

    # Consulta SPARQL ajustada
    query = f"""
    SELECT ?property ?propertyLabel ?value ?valueLabel
    WHERE {{
        wd:{entity_id} ?property ?value.
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(sparql_endpoint, params={'query': query, 'format': 'json'}, headers=headers)
    data = response.json()

    # Diccionario para almacenar la información relevante
    properties_dict = {
    "http://www.wikidata.org/prop/direct/P31":"instance of",
    "http://www.wikidata.org/prop/direct/P123":"publisher",
    "http://www.wikidata.org/prop/direct/P178":"developer",
    "http://www.wikidata.org/prop/direct/P577":"publication date",
    }

    # Diccionario donde almacenaremos la información filtrada
    game_wikidata = {}

    # Recorrer los resultados
    for result in data['results']['bindings']:
        property_uri = result['property']['value']

        # Verificar si la propiedad está en properties_dict
        if property_uri in properties_dict:
            property_label = properties_dict[property_uri]
            # Si existe el valor en 'valueLabel', lo usamos; de lo contrario, usamos el valor crudo
            value_label = result.get('valueLabel', {}).get('value', result['value']['value'])

            # Guardamos la información en el diccionario game_wikidata
            game_wikidata[property_label] = value_label

    # Imprimir el diccionario resultante
    print(game_wikidata)
    return game_wikidata

# Consulta el artículo con ID Q56759854
game_wikidata = get_wikidata_info("Q56759854")


{'instance of': 'board game', 'publisher': 'HUCH!', 'developer': 'Inka and Markus Brand', 'publication date': '2017-01-01T00:00:00Z'}


### Games with same publisher ###

In [19]:
import requests

def get_items_by_huch_uri():
    sparql_endpoint = "https://query.wikidata.org/sparql"

    # Consulta SPARQL usando el URI completo de P123 (editorial) y Q1566108 (HUCH!)
    query = """
    SELECT ?item ?itemLabel ?publisherLabel
    WHERE {
      ?item <http://www.wikidata.org/prop/direct/P123> <http://www.wikidata.org/entity/Q1566108>.  # HUCH! como editorial
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(sparql_endpoint, params={'query': query, 'format': 'json'}, headers=headers)
    data = response.json()

    # Imprimir los resultados
    same_publisher = []
    if data['results']['bindings']:
        for result in data['results']['bindings']:
            item_label = result['itemLabel']['value']
            publisher_label = result.get('publisherLabel', {}).get('value', "Unknown Publisher")
            item_id = result['item']['value']
            print(f"Item: {item_label}, ID: {item_id}")
            same_publisher.append(item_label)
    else:
        print("No results found for HUCH! publisher.")
    return same_publisher

# Ejecutar la consulta
same_publisher = get_items_by_huch_uri()


Item: YINSH, ID: http://www.wikidata.org/entity/Q2626490
Item: Ta-Ke, ID: http://www.wikidata.org/entity/Q45017381
Item: Cookies, ID: http://www.wikidata.org/entity/Q50673606
Item: Djinn, ID: http://www.wikidata.org/entity/Q56640403
Item: Rajas of the Ganges, ID: http://www.wikidata.org/entity/Q56759854
Item: Q60983282, ID: http://www.wikidata.org/entity/Q60983282
Item: Nut Job, ID: http://www.wikidata.org/entity/Q61092606
Item: Espresso Doppio, ID: http://www.wikidata.org/entity/Q111378799


### Games with same developer ###

In [20]:
import requests

def get_items_by_developer_uri():
    sparql_endpoint = "https://query.wikidata.org/sparql"

    # Consulta SPARQL usando el URI completo de P178 (desarrollador) y Q1663646 (Inka and Markus Brand)
    query = """
    SELECT ?item ?itemLabel ?developerLabel
    WHERE {
      ?item <http://www.wikidata.org/prop/direct/P178> <http://www.wikidata.org/entity/Q1663646>.  # Inka and Markus Brand como desarrollador
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(sparql_endpoint, params={'query': query, 'format': 'json'}, headers=headers)
    data = response.json()

    # Imprimir los resultados
    same_developer = []
    if data['results']['bindings']:
        for result in data['results']['bindings']:
            item_label = result['itemLabel']['value']
            developer_label = result.get('developerLabel', {}).get('value', "Unknown Developer")
            item_id = result['item']['value']
            print(f"Item: {item_label}, ID: {item_id}")
            same_developer.append(item_label)
    else:
        print("No results found for Inka and Markus Brand as developer.")
    return same_developer

# Ejecutar la consulta
same_developer = get_items_by_developer_uri()


Item: Village, ID: http://www.wikidata.org/entity/Q2526032
Item: Burg der 1000 Spiegel, ID: http://www.wikidata.org/entity/Q44660156
Item: Monster Trap, ID: http://www.wikidata.org/entity/Q44660479
Item: Rajas of the Ganges, ID: http://www.wikidata.org/entity/Q56759854


# Step 2: Classifier #

## Random Forest Model##

In [28]:
import pandas as pd

# Define categories and prompts explicitly
mock_data = [
    # Rules category
    ("What are the rules for placing workers in Rajas of the Ganges?", "rules"),
    ("How does the river mechanic work in Rajas of the Ganges?", "rules"),
    ("Can you explain the scoring system in Rajas of the Ganges?", "rules"),
    ("What happens if two players tie in Rajas of the Ganges?", "rules"),
    ("How do I gain karma in Rajas of the Ganges?", "rules"),
    ("What are the rules for using dice in Rajas of the Ganges?", "rules"),
    ("How does a player win in Rajas of the Ganges?", "rules"),
    ("What is the setup process for Rajas of the Ganges?", "rules"),
    ("How do I use the bonus tiles in Rajas of the Ganges?", "rules"),
    ("Can I gain points through the trade mechanism in Rajas of the Ganges?", "rules"),
    ("How does the taxation mechanic work in Rajas of the Ganges?", "rules"),
    ("What do the different colors of dice mean in Rajas of the Ganges?", "rules"),
    ("Can you explain how the river and palace tracks are connected?", "rules"),
    ("How do I get to use the special actions in Rajas of the Ganges?", "rules"),
    ("What are the different types of buildings in Rajas of the Ganges?", "rules"),
    ("How does the market mechanic work in Rajas of the Ganges?", "rules"),
    ("What is the role of the player board in Rajas of the Ganges?", "rules"),
    ("How do I earn money in Rajas of the Ganges?", "rules"),
    ("What do the resources in Rajas of the Ganges represent?", "rules"),
    ("How does the prestige track work in Rajas of the Ganges?", "rules"),
    ("What is the importance of the river in Rajas of the Ganges?", "rules"),
    ("How do you activate special abilities in Rajas of the Ganges?", "rules"),
    ("How do I gain points from the karma track in Rajas of the Ganges?", "rules"),
    ("What happens if I go over the maximum number of workers?", "rules"),
    ("How do I use the dice placement in Rajas of the Ganges?", "rules"),
    ("How does the taxation mechanic work in Rajas of the Ganges?", "rules"),
    ("Can I choose which dice to place in Rajas of the Ganges?", "rules"),
    ("How do I gain victory points in Rajas of the Ganges?", "rules"),
    ("What is the importance of the palace in Rajas of the Ganges?", "rules"),
    ("How do I use the river and palace tracks in Rajas of the Ganges?", "rules"),

    # Review category
    ("What do players think of Rajas of the Ganges?", "review"),
    ("Is Rajas of the Ganges a good game for beginners?", "review"),
    ("What are the pros and cons of Rajas of the Ganges?", "review"),
    ("Why do people like Rajas of the Ganges?", "review"),
    ("How does Rajas of the Ganges compare to other worker placement games?", "review"),
    ("Is Rajas of the Ganges worth buying?", "review"),
    ("What makes Rajas of the Ganges stand out among worker placement games?", "review"),
    ("Is Rajas of the Ganges considered a thematic game?", "review"),
    ("What is the player experience like in Rajas of the Ganges?", "review"),
    ("How does Rajas of the Ganges handle player interaction?", "review"),
    ("Is Rajas of the Ganges a good game for strategy enthusiasts?", "review"),
    ("What makes Rajas of the Ganges a fun game to play repeatedly?", "review"),
    ("How is the replayability of Rajas of the Ganges?", "review"),
    ("Is Rajas of the Ganges more luck-based or strategy-based?", "review"),
    ("What kind of player would enjoy Rajas of the Ganges?", "review"),
    ("How would you rate the components of Rajas of the Ganges?", "review"),
    ("What makes the river track mechanic interesting in Rajas of the Ganges?", "review"),
    ("How does the game balance strategy and randomness?", "review"),
    ("Is Rajas of the Ganges suitable for solo play?", "review"),
    ("How long does a typical game of Rajas of the Ganges last?", "review"),
    ("Is Rajas of the Ganges a good game for small groups?", "review"),
    ("How does the game compare to other games by the same designer?", "review"),
    ("What makes the karma track interesting in Rajas of the Ganges?", "review"),
    ("How does the dice mechanic influence gameplay in Rajas of the Ganges?", "review"),
    ("What is the difficulty level of Rajas of the Ganges?", "review"),
    ("How does Rajas of the Ganges compare to other medium-weight games?", "review"),
    ("Is Rajas of the Ganges a game that rewards long-term planning?", "review"),
    ("How accessible are the rules of Rajas of the Ganges?", "review"),

    # Stats category
    ("What is the average playtime of Rajas of the Ganges?", "stats"),
    ("How many players can play Rajas of the Ganges?", "stats"),
    ("What is the ideal player count for Rajas of the Ganges?", "stats"),
    ("What is the current ranking of Rajas of the Ganges on BoardGameGeek?", "stats"),
    ("How difficult is Rajas of the Ganges to learn?", "stats"),
    ("How long does it take to set up Rajas of the Ganges?", "stats"),
    ("What is the average duration of a game of Rajas of the Ganges?", "stats"),
    ("What is the weight of Rajas of the Ganges on BoardGameGeek?", "stats"),
    ("What is the player count range for Rajas of the Ganges?", "stats"),
    ("How long do games typically last with two players?", "stats"),
    ("What is the best player count for Rajas of the Ganges?", "stats"),
    ("How does player count affect the gameplay experience?", "stats"),
    ("What is the setup time for Rajas of the Ganges?", "stats"),
    ("What is the age recommendation for Rajas of the Ganges?", "stats"),
    ("What is the minimum player count for Rajas of the Ganges?", "stats"),
    ("How much downtime is there between turns in Rajas of the Ganges?", "stats"),
    ("What is the skill level required to play Rajas of the Ganges?", "stats"),
    ("What is the time required to explain the rules of Rajas of the Ganges?", "stats"),
    ("How much time does it take to complete a solo game of Rajas of the Ganges?", "stats"),
    ("How does the game scale with different player counts?", "stats"),
    ("What is the maximum number of players for Rajas of the Ganges?", "stats"),
    ("What is the minimum age requirement for Rajas of the Ganges?", "stats"),
    ("What is the total number of components in Rajas of the Ganges?", "stats"),
    ("How many different types of workers are there in Rajas of the Ganges?", "stats"),

    # Related Games category
    ("What games are similar to Rajas of the Ganges?", "related_games"),
    ("Can you recommend games like Rajas of the Ganges?", "related_games"),
    ("Are there any other games by the same designers as Rajas of the Ganges?", "related_games"),
    ("How does Rajas of the Ganges compare to Agricola?", "related_games"),
    ("Is Rajas of the Ganges similar to Stone Age?", "related_games"),
    ("What games are good follow-ups to Rajas of the Ganges?", "related_games"),
    ("How does Rajas of the Ganges compare to Le Havre?", "related_games"),
    ("What is the difference between Rajas of the Ganges and Tigris & Euphrates?", "related_games"),
    ("How does Rajas of the Ganges differ from Puerto Rico?", "related_games"),
    ("Is Rajas of the Ganges similar to Concordia?", "related_games"),
    ("Can you compare Rajas of the Ganges to Imperial Settlers?", "related_games"),
    ("Is Rajas of the Ganges a good alternative to Caverna?", "related_games"),
    ("How does Rajas of the Ganges compare to the game Clans of Caledonia?", "related_games"),
    ("What games have similar mechanics to Rajas of the Ganges?", "related_games"),
    ("How does Rajas of the Ganges compare to The Estates?", "related_games"),
    ("What are other worker placement games similar to Rajas of the Ganges?", "related_games"),
    ("How does Rajas of the Ganges compare to other strategy games?", "related_games"),
    ("Can you suggest other games that combine resource management and worker placement?", "related_games"),
    ("How does Rajas of the Ganges compare to Castles of Burgundy?", "related_games"),
    ("What other games feature a river mechanic like Rajas of the Ganges?", "related_games"),
    ("What games have similar dice mechanics to Rajas of the Ganges?", "related_games"),
    ("Are there other games with karma as a mechanic like Rajas of the Ganges?", "related_games"),
    ("How does Rajas of the Ganges compare to Great Western Trail?", "related_games"),
    ("What are some games that focus on economy and expansion like Rajas of the Ganges?", "related_games"),
    ("How does Rajas of the Ganges compare to Gaia Project?", "related_games")
]

# Create the DataFrame
training_data = pd.DataFrame(mock_data, columns=["prompt", "type"])

# Display the DataFrame
training_data

,prompt,type
0,What are the rules for placing workers in Raja...,rules
1,How does the river mechanic work in Rajas of t...,rules
2,Can you explain the scoring system in Rajas of...,rules
3,What happens if two players tie in Rajas of th...,rules
4,How do I gain karma in Rajas of the Ganges?,rules
...,...,...
102,What games have similar dice mechanics to Raja...,related_games
103,Are there other games with karma as a mechanic...,related_games
104,How does Rajas of the Ganges compare to Great ...,related_games
105,What are some games that focus on economy and ...,related_games


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Convertir las preguntas (prompts) en vectores de características usando TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(training_data['prompt'])

# Las etiquetas son las categorías
y = training_data['type']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo de Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Evaluar el modelo
y_pred = random_forest.predict(X_test)

# Mostrar las estadísticas de evaluación
print(classification_report(y_test, y_pred))


               precision    recall  f1-score   support

related_games       0.75      1.00      0.86         3
       review       0.50      0.33      0.40         6
        rules       0.55      1.00      0.71         6
        stats       0.33      0.14      0.20         7

     accuracy                           0.55        22
    macro avg       0.53      0.62      0.54        22
 weighted avg       0.49      0.55      0.48        22



In [30]:
# TESTING
prompt = ["What is the importance of karma in Rajas of the Ganges?"]
embedded = vectorizer.transform(prompt)
prediction = random_forest.predict(embedded)
print("Prediction:", prediction[0])

prompt = ["How many people plays the game?"]
embedded = vectorizer.transform(prompt)
prediction = random_forest.predict(embedded)
print("Prediction:", prediction[0])


Prediction: rules
Prediction: review


## LLM Model ##

In [34]:
types=["rules", "review", "stats", "related_games"]

from huggingface_hub import InferenceClient
from google.colab import userdata

client = InferenceClient(api_key=userdata.get('HF_TOKEN'))
role="user"
content=f"""
You are an assistant that will help end users with a board game called Rajas of the Ganges.
You will receive a prompt (ideally a question) and you should classify it in one of this four categories.
rules: If the prompt is related to the rules of the game.
review: If the prompt is related to game reviews, opinions or comparisons.
stats: If the prompt is related to the statistics of the game, such as strategy rank, amount of players.
related_games: If the prompt asks for games related to Rajas of the Ganges. For example, for sharing publisher or developer.
Your response is one and only one of these categories, without any other context or additional comments.
This is the prompt:
"""

def llm_classifier(prompt):
  message=[{"role": role, "content": content+prompt}]
  completion = client.chat.completions.create(model="Qwen/Qwen2.5-72B-Instruct", messages=message, max_tokens=10)
  classification = completion.choices[0].message.content.strip()
  return classification

print(llm_classifier("What other games feature a river mechanic like Rajas of the Ganges?"))

related_games


In [37]:
sum=0
for i in range(training_data.shape[0]):
  llm_prediction = llm_classifier(training_data["prompt"][i])
  if llm_prediction==training_data["type"][i]:
    sum+=1

print(sum/training_data.shape[0])

0.7102803738317757


# Step 2: Language Managing #

In [ ]:
from langdetect import detect
def isSpanish(input):
  idioma = detect(input)
  if idioma=="es":
    return True
  elif idioma=="en":
    return False
  else:
    return None

print(isSpanish("Hola, ¿cómo estás?"))
print(isSpanish("Hello, how are you?"))


from deep_translator import GoogleTranslator
translated = GoogleTranslator(source='en', target='es').translate('I want to translate this text')
print(translated)


def translate_to_english(content):
  if type(content)==str:
    translated = GoogleTranslator(source='es', target='en').translate(content)
    return translated
  else:
    return None

def translate_to_spanish(content):
  if type(content)==str:
    translated = GoogleTranslator(source='en', target='es').translate(content)
    return translated
  else:
    return None

print(translate_to_english("Hola, mi nombre es Francisco."))
print(translate_to_spanish("Hello, my name is Francisco."))

# Step 3: RAG #